In [1]:
import openai
import psycopg2
import pandas as pd
from sentence_transformers import SentenceTransformer
import requests, uuid, json, re, time

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
openai.api_type = "azure"
openai.api_base = "https://fusion-gpt-training.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "a83037e20d3d4d7982b0c52080e730fb"

In [3]:
host = "pureelawvec.postgres.database.azure.com"
dbname = "postgres"
user = "pgadmin"
password = "Puree1234"
sslmode = "require"
# Construct connection string

In [5]:
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")
cursor = conn.cursor()
# Fetch all rows from table

Connection established


In [51]:
x = pd.read_csv("output (6).csv")
x['Question'] = x['Question'].str.replace('Question : ', '')
x['Answer'] = x['Answer'].str.replace('Answer : ', '')
x

,Question,Answer
0,"""โดยทุจริต"" ในประมวลกฎหมายหมายถึงอะไร?","""โดยทุจริต"" ในประมวลกฎหมายหมายความว่า เพื่อแสว..."
1,"""ทางสาธารณะ"" ในประมวลกฎหมายหมายถึงอะไร?","""ทางสาธารณะ"" หมายความว่า ทางบกหรือทางน้ำสำหรับ..."
2,"""สาธารณสถาน"" ในประมวลกฎหมายหมายถึงอะไร?","""สาธารณสถาน"" หมายความว่า สถานที่ใด ๆ ซึ่งประชา..."
3,"""เคหสถาน"" ในประมวลกฎหมายหมายถึงอะไร?","""เคหสถาน"" หมายความว่า ที่ซึ่งใช้เป็นที่อยู่อาศ..."
4,"""อาวุธ"" ในประมวลกฎหมายหมายถึงอะไร?","""อาวุธ"" หมายความรวมถึงสิ่งซึ่งไม่เป็นอาวุธโดยส..."
...,...,...
840,ถ้าฉันรู้ว่ามีคนจะเป็นกบฏแต่ไม่ได้ทำอะไรเพื่อช...,ไม่ใช่ ตามมาตรา 114 ต้องมีการกระทำในฐานะที่ช่ว...
841,ถ้าฉันสมคบกับคนอื่นๆ เพื่อเป็นกบฏแต่ยังไม่ได้ก...,ใช่ ตามมาตรา 114 การสมคบกันเพื่อเป็นกบฏหรือกระ...
842,ถ้าฉันสะสมกำลังพลแต่ไม่ได้สมคบกับคนอื่นๆ ฉันจะ...,ไม่ใช่ ตามมาตรา 114 การสะสมกำลังพลต้องเป็นส่วน...
843,ถ้าฉันยุยงราษฎรให้เป็นกบฏแต่ไม่ได้สมคบกับคนอื่...,ใช่ มาตรา 114 ระบุว่าผู้ใดยุยงราษฎรให้เป็นกบฏ ...


In [5]:
Lawyer_prompt = "LawBot, a GPT-based AI assistant designed specifically for answering legal questions and explaining complex legal terms.\
Analysis the Question sector and provide a comprehensive and briefly answer in Answer section.The answer must address all aspects\
of the Question and support your argument with appropriate legal references in Context section. Only answered by thai language only "


In [ ]:
from nltk.translate.bleu_score import corpus_bleu  
  
# Example reference and generated texts  
references = x.Answer  
hypotheses = x.Answer  
  
# Calculate BLEU score  
bleu_score = corpus_bleu(references, hypotheses)  
  
# Print the BLEU score  
print("BLEU score:", bleu_score)  


In [10]:
import openai  # Make sure to import the OpenAI library

class Chatbot():
    def __init__(self):
        self.memory = [{"role": "system", "content": ""}, {"role": "user", "content": ""}]
        self.Lawyer_prompt = "LawBot, a GPT-based AI assistant designed specifically for answering legal questions and explaining complex legal terms.\
        Analysis the Question sector and provide a comprehensive and briefly answer in Answer section. Only answered by thai language only "
    
    def thai2eng(self ,input):
        key = "041e6b79b96146eabafaf524d78911be"
        endpoint = "https://api.cognitive.microsofttranslator.com"
        location = "southeastasia"
        path = '/translate'
        constructed_url = endpoint + path
        params = {
            'api-version': '3.0',
            'from': 'th',
            'to': ['en']
        }
        headers = {
            'Ocp-Apim-Subscription-Key': key,
            'Ocp-Apim-Subscription-Region': location,
            'Content-type': 'application/json',
            'X-ClientTraceId': str(uuid.uuid4())
        }
        body = [{
            'text': input
        }]
        request = requests.post(constructed_url, params=params, headers=headers, json=body)
        response = request.json()
        return response[0]["translations"][0]['text']

    def text2vec(self, input):
        embed_list = []
        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        embeddings = model.encode(input)
        embed_list.append(embeddings.tolist())
        return embed_list[0]

    def search_vectordb(self, question):
        search_text = []
        try:
            cursor.execute(f"""SELECT text_thai, cosine_distance('{self.text2vec(question)}', embedding) as cosign
            FROM Criminal_Law
            ORDER BY cosine_distance('{self.text2vec(question)}', embedding) ASC LIMIT 1
            """)
            query = cursor.fetchall()
            text_thai_results = [row[0] for row in query] 
            return text_thai_results
        except Exception as e:  
                print(f"Error: {e}")
                conn.rollback()
        return "Not found any related."

    def gpt3_sum(self, user_input, engine='gpt-4', temp=0.5, top_p=0.95, tokens=2000, freq_pen=0.0, pres_pen=0.2):
        retry, max_retry = 0, 5
        while retry < max_retry:
            try:
                setting = [{"role": "system", "content": "Summary the user input as a bullet, use only Thai language"}, 
                        {"role": "user", "content": f"{user_input}"}]
                response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
                                                        top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen,
                                                        messages=setting)
                return response['choices'][0]['message']['content']
            except openai.error.OpenAIError as e:
                print(f"Error: {e}")
                retry += 1
                if retry < max_retry:
                    print(f"Due to content filtering, retrying. (attempt {retry}/{max_retry})...")
                    time.sleep(3)  # Add a delay before retrying
                else:
                    print("Max retries reached. Exiting. Please change the question")
                    break
        return response['choices'][0]['message']['content']

    def chat(self, user_input, engine='gpt-4-32k', temp=0.5, top_p=0.95, tokens=4000, freq_pen=0.0, pres_pen=0.2):
        retry, max_retry = 0, 5
        while retry < max_retry:
            try:
                # user_input = input("User:")
                if user_input == "stop":
                    retry += max_retry
                    break
                self.clear_memory()
                self.update_memory('user', user_input)
                lawyer_prompt_content = self.Lawyer_prompt + ''.join(self.search_vectordb(self.thai2eng(user_input)))
                self.update_memory('system', lawyer_prompt_content)
                response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
                                                        top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen,
                                                        messages=self.memory)
                # self.update_memory('system', "This is memory section that keep old conversation <memory>{mem}</memory>"
                #     .format(mem = response['choices'][0]['message']['content']))
                print("LawChatBot:", response['choices'][0]['message']['content'])
                return response['choices'][0]['message']['content']

            except openai.error.OpenAIError as e:
                print(f"Error: {e}")
                retry += 1
                if retry < max_retry:
                    print(f"Due to content filtering, retrying. (attempt {retry}/{max_retry})...")
                    time.sleep(3)  # Add a delay before retrying
                else:
                    print("Max retries reached. Exiting. Please change the question")
                    break
                
    def update_memory(self, role, content):
        self.memory.append({"role": role, "content": content})
        
    def clear_memory(self):
        self.memory = [{"role": "system", "content": ""}, {"role": "user", "content": ""}]


In [58]:
bot = Chatbot()  

In [59]:
#ขโมยของผิดไหม
#ฆ่าคนมีความผิดไหม
answer  = []
for idx in range(20):
    ret = bot.chat(x.Question.iloc[idx])
    answer.append(ret)

LawChatBot: "โดยทุจริต" ในประมวลกฎหมายหมายถึงการกระทำที่เกิดจากความตั้งใจที่ผิดกฎหมายหรือมีเจตนาทำอันตรายต่อผู้อื่น ซึ่งอาจรวมถึงการหลอกลวง, การฉ้อโกง, หรือการกระทำที่ไม่ซื่อสัตย์ และมักจะเกี่ยวข้องกับการทำลายสิทธิ์, ความไว้วางใจ, หรือสวัสดิภาพของผู้อื่น.
LawChatBot: "ทางสาธารณะ" ในประมวลกฎหมายหมายถึงทางบกหรือทางน้ำที่ใช้สำหรับการขนส่งสาธารณะ รวมถึงทางรถไฟหรือรถรางสำหรับการขนส่งสาธารณะด้วย
LawChatBot: "สาธารณสถาน" ในประมวลกฎหมายหมายถึงสถานที่หรืออาคารที่เปิดให้ประชาชนใช้งานหรือเข้ามาใช้บริการ ไม่ว่าจะเป็นโรงพยาบาล สถานศึกษา สถานีรถไฟ สนามบิน สถานที่ท่องเที่ยว ห้องสมุด ฯลฯ และยังรวมถึงสถานที่สาธารณะที่เป็นที่สาธารณะใช้หรือเข้าถึงได้ ไม่ว่าจะเป็นทางจราจร สวนสาธารณะ หรือที่ว่างๆ ในตัวเมือง ฯลฯ
LawChatBot: "เคหสถาน" ในประมวลกฎหมายหมายถึง "ที่ทำการ" หรือ "สถานที่ประกอบการ" ซึ่งสามารถหมายถึงสถานที่ที่ใช้ในการดำเนินธุรกิจ, ที่ทำการของรัฐบาล, สถานที่ที่ใช้ในการผลิตหรือสถานที่อื่น ๆ ที่ใช้ในการดำเนินกิจการ.
LawChatBot: "อาวุธ" ในประมวลกฎหมายหมายถึง วัตถุซึ่งมิใช่อาวุธโดยธรรมชาติ แต่ใช้หรือมีเจต

In [32]:
conn.commit()
cursor.close()
conn.close()

In [60]:
from scipy.spatial.distance import cosine  
cumu = 0
for idx in range(20):
    ans = Chatbot().text2vec(answer[idx])
    ques = Chatbot().text2vec((x.Question.iloc[idx]))
    cumu +=  1 - cosine(ques, ans)  
  
# Print the cosine similarity  
print(f"Cosine similarity: {(cumu/20).round(2)}")  


Cosine similarity: 0.6946368392373813


In [57]:
answer

['"โดยทุจริต" ในประมวลกฎหมายหมายถึงการกระทำที่มีเจตนาที่ไม่ซื่อสัตย์หรือไม่สุจริต เช่น การหลอกลวง, การฉ้อโกง, การกระทำในทางที่ผิดกฎหมายเพื่อให้ได้ประโยชน์ในทางที่ไม่เป็นธรรม. สิ่งเหล่านี้สามารถเกิดขึ้นในหลากหลายสถานการณ์ ไม่ว่าจะเป็นการทำธุรกรรมทางการเงิน, การทำธุรกิจ, หรือการปฏิบัติตามหน้าที่ของตำแหน่งหน้าที่.',
 '"ทางสาธารณะ" ในประมวลกฎหมายหมายถึง ทางบกหรือทางน้ำที่ใช้สำหรับการขนส่งสาธารณะ รวมถึงทางรถไฟหรือรถรางที่ใช้สำหรับการขนส่งสาธารณะ',
 '"สาธารณสถาน" ตามประมวลกฎหมายหมายถึงสถานที่ที่เปิดให้ประชาชนเข้ามาใช้งานหรือเข้ามาติดต่อ เช่น สถานีรถไฟ สถานีรถเมล์ สนามบิน โรงเรียน สนามกีฬา สวนสาธารณะ ห้องสมุด โรงพยาบาล ฯลฯ สาธารณสถานเหล่านี้เป็นสถานที่ที่ประชาชนมีสิทธิ์เข้าใช้งานโดยไม่ต้องได้รับอนุญาตจากเจ้าของหรือผู้ดูแลสถานที่แต่อยู่ภายใต้ข้อจำกัดและกฎระเบียบที่กำหนดโดยเจ้าของหรือผู้ดูแลสถานที่ และ/หรือ กฎหมายที่เกี่ยวข้อง',
 '"เคหสถาน" ในประมวลกฎหมายหมายถึง "สถานที่" หรือ "สถานที่ตั้ง" ซึ่งอาจรวมถึงอาคาร, บ้าน, สิ่งก่อสร้าง, หรือพื้นที่ที่มีการใช้งานหรือมีกิจกรรมใดๆ อยู่ มันเป็นคำที่ใช้ในท